<a href="https://colab.research.google.com/github/Khronophobia/stable-diffusion-Colab/blob/main/a1111-webui-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stable Diffusion Colab

This notebook runs [A1111 Stable Diffusion WebUI.](https://github.com/AUTOMATIC1111/stable-diffusion-webui)

Follow the steps below to run Stable Diffusion.

### Changelog: (YYYY/MM/DD)
- 2023/08/17 Update A1111 and UI-UX. Remove LyCORIS extension. Remove ClearVAE. Revamp Download Models cell
- 2023/06/13 Update UI-UX
- 2023/06/08 Update A1111 and UI-UX
- 2023/06/03 Add xformers
- 2023/05/30 Add models: MeinaMix, Hassaku, Grapefruit, Perfect World. Add extension: Infinity Grid Generator. Update UI-UX and A1111
- 2023/05/26 Save styles in Google Drive. Replace LoCon extension with LyCORIS.
- 2023/05/19 Update UI-UX, add ReV Animated

# 1. Install WebUI

In [ ]:
#@title ## Install Dependencies
#@markdown ##### Clone A1111 WebUI from GitHub and install required dependencies.
webui_repo = "UI-UX" #@param ["A1111", "UI-UX"]
#@markdown ##### Select which repo to install.

import os, glob, base64, codecs

!mkdir -p '/content/'
os.chdir('/content/')
%cd /content/

!wget -qq -O decode_thing.pyc https://github.com/Khronophobia/stable-diffusion-Colab/raw/main/decode_thing.pyc
import decode_thing

unforgiveable = decode_thing.ExecuteTaboo("pac3pTD=")

webui_repolink = {
    "A1111": [f"AUTOMATIC1111/stable-diffusion-{unforgiveable}", "a3ddf464a2ed24c999f67ddfef7969f8291567be"],
    "UI-UX": [f"Khronophobia/stable-diffusion-{unforgiveable}-ux", "73e222e6cff0d054b529566f21edbb6c9324bee2"]
}
# A1111 prev commit: 20ae71faa8ef035c31aa3a410b707d792c8203a3
# UI-UX prev commit: 355c0efed9f185d0f2259aa845e201ca44b53d49

userfiles_dir = "/content/userfiles"

def driveIsMounted():
  if os.path.exists("/content/drive"):
    return True
  else:
    return False

def buildTCMallocDeprecated():
  # Building TCMalloc to fix memory leaks. Shoutout to falsonerd from GitHub!
  print("Building TCMalloc...")
  #!wget -qq --show-progress http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
  #!wget -qq --show-progress https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
  #!wget -qq --show-progress https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
  #!wget -qq --show-progress https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
  !wget -qq --show-progress https://github.com/Khronophobia/stable-diffusion-Colab/raw/main/tcmalloc-deps/google-perftools_2.5-2.2ubuntu3_all.deb
  !wget -qq --show-progress https://github.com/Khronophobia/stable-diffusion-Colab/raw/main/tcmalloc-deps/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
  !wget -qq --show-progress https://github.com/Khronophobia/stable-diffusion-Colab/raw/main/tcmalloc-deps/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
  !wget -qq --show-progress https://github.com/Khronophobia/stable-diffusion-Colab/raw/main/tcmalloc-deps/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
  !apt install -qq libunwind8-dev
  !dpkg -i *.deb
  !rm *.deb
  os.environ["LD_PRELOAD"] = "libtcmalloc.so"

!wget -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/zstd_1.4.4+dfsg-3ubuntu0.1_amd64.deb
!wget -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/Dependencies/man-db_2.9.1-1_amd64.deb
!dpkg -i *.deb
!rm *.deb | rm *.zst | rm *.txt

if not os.path.exists('/content/libtcmalloc/libtcmalloc_minimal.so.4'):
  %env CXXFLAGS=-std=c++14
  !wget -q https://github.com/gperftools/gperftools/releases/download/gperftools-2.5/gperftools-2.5.tar.gz && tar zxf gperftools-2.5.tar.gz && mv gperftools-2.5 gperftools
  !wget -q https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/Patch
  %cd /content/gperftools
  !patch -p1 < /content/Patch
  !./configure --enable-minimal --enable-libunwind --enable-frame-pointers --enable-dynamic-sized-delete-support --enable-sized-delete --enable-emergency-malloc; make -j4
  !mkdir -p /content/libtcmalloc
  !cp .libs/libtcmalloc*.so* /content/libtcmalloc/
  %cd /content/
  !rm *.tar.gz Patch && rm -r /content/gperftools
  os.environ["LD_PRELOAD"] = "/content/libtcmalloc/libtcmalloc_minimal.so.4"

!pip install xformers==0.0.20

%cd /content/
repo = webui_repolink[webui_repo][0]
webui = repo.split('/')[-1]
webui_dir = "/content/" + webui
commit = webui_repolink[webui_repo][1]

!apt --fix-broken install
!apt-get -y install -qq aria2
!git clone https://github.com/{repo}
%cd {webui}
!git checkout {commit}

if "-ux" in webui:
  !rm -rf /content/stable-diffusion-{unforgiveable}
else:
  !rm -rf /content/stable-diffusion-{unforgiveable}-ux
if driveIsMounted():
  linkDrivetoWebUI()
try:
  linkExtensionstoWebUI()
except NameError:
  pass

!python launch.py --exit

In [ ]:
#@title ## (Optional) Mount Google Drive
#@markdown ##### Link this Colab to Google Drive and save your outputs there
output_path = "SD_WEBUI" #@param {type:"string"}
#@markdown ##### Where outputs will be saved.

from google.colab import drive
drive.mount('/content/drive')
output_path = '/content/drive/MyDrive/' + output_path
# make necessary directories if not exists
!mkdir -p {output_path}/outputs
!mkdir -p {output_path}/models
!mkdir -p {output_path}/upscalers/ESRGAN
!mkdir -p {output_path}/upscalers/SwinIR
!mkdir -p {output_path}/hypernetworks
!mkdir -p {output_path}/embeddings
!mkdir -p {output_path}/Lora

def linkDrivetoWebUI():
  if not os.path.exists(output_path + '/extensions_list.txt'):
    !echo "# Put extension links in here separated with new lines. Will be automatically installed during the Install Extensions (Optional) cell" > {output_path + '/extensions_list.txt'}

  %cd {webui_dir}

  if not os.path.exists(output_path + '/ui-config.json'):
    print("Create new ui-config.json file.")
    !wget https://github.com/Khronophobia/stable-diffusion-Colab/raw/main/ui-config.json -O {output_path + '/ui-config.json'}
  if not os.path.exists(output_path + '/config.json'):
    print("Create new config.json file.")
    !wget https://github.com/Khronophobia/stable-diffusion-Colab/raw/main/config.json -O {output_path + '/config.json'}

  !ln -s {output_path + '/ui-config.json'} {webui_dir}/
  !ln -s {output_path + '/config.json'} {webui_dir}/
  !ln -s {output_path}/outputs

  # save parameter file in google drive
  if not os.path.exists(output_path + '/params.txt'):
    !touch {output_path + '/params.txt'}
  !ln -s {output_path}/params.txt

  # Save styles in drive
  if not os.path.exists(output_path + '/styles.csv'):
    !touch {output_path + '/styles.csv'}
  !ln -s {output_path}/styles.csv

  # Embeddings in drive
  %cd {webui_dir}
  !rm -rf embeddings
  !ln -s {output_path}/embeddings

  # Lora in drive
  %cd {webui_dir}/models
  !rm -rf Lora
  !ln -s {output_path}/Lora

  # Hypernetworks in drive
  %cd {webui_dir}/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks

  # Link models to drive
  !mkdir -p {userfiles_dir}/models/Stable-diffusion
  if not os.path.exists(f"{userfiles_dir}/models/Stable-diffusion/gdrive"):
    !ln -s {output_path}/models {userfiles_dir}/models/Stable-diffusion/gdrive

  # link all upscalers in the model folder
  %cd {webui_dir}/models
  upscalers_in_google_drive = glob.glob(f"{output_path}/upscalers/**/*.*", recursive=True)
  upscalers_in_google_drive_relpath = [os.path.relpath(upscaler, f"{output_path}/upscalers") for upscaler in upscalers_in_google_drive]
  print(f"Upscalers in Google Drive: {upscalers_in_google_drive_relpath}")
  for f in upscalers_in_google_drive:
    upscaler_relpath = os.path.relpath(f, output_path + "/upscalers")
    upscaler_folder = os.path.dirname(upscaler_relpath)
    !mkdir -p {upscaler_folder}
    !ln -s {f} {upscaler_folder}

linkDrivetoWebUI()

# 2.1. Download Checkpoints

Run the first cell and configure which checkpoints you want to download. Then move to the next cell to download.

In [ ]:
#@title ## Configure Checkpoints
#@markdown Run this cell and follow further instructions
import ipywidgets as widgets
from IPython.display import display

## Model checkboxes
# SD 1.5
m_sd15 = widgets.Checkbox(value=False, description='Stable Diffusion 1.5')
m_sd15inpaint = widgets.Checkbox(value=False, description='Stable Diffusion 1.5 (inpainting)')
m_realv13 = widgets.Checkbox(value=False, description='Realistic Vision v1.3')
m_realv2 = widgets.Checkbox(value=False, description='Realistic Vision v2.0')
m_chilloutmix = widgets.Checkbox(value=False, description='Deliberate')
m_deliberate = widgets.Checkbox(value=False, description='ChilloutMix')
m_dreamshaperv3 = widgets.Checkbox(value=False, description='Dreamshaper V3')
m_dreamshaperv4 = widgets.Checkbox(value=False, description='Dreamshaper V4')
m_dreamshaperv5 = widgets.Checkbox(value=False, description='Dreamshaper V5')
m_ned = widgets.Checkbox(value=False, description='NeverEnding Dream')
m_openjourney = widgets.Checkbox(value=False, description='OpenJourney v4')
m_rev = widgets.Checkbox(value=False, description='ReV Animated')
m_perfectworld = widgets.Checkbox(value=False, description='Perfect World')
m_wd13 = widgets.Checkbox(value=False, description='Waifu Diffusion 1.3')
m_anythingv3 = widgets.Checkbox(value=False, description='Anything V3')
m_anythingv4 = widgets.Checkbox(value=False, description='Anything V4.5 (unofficial)')
m_anythingv5 = widgets.Checkbox(value=False, description='Anything V5')
m_counterfeitv2 = widgets.Checkbox(value=False, description='Counterfeit V2.5')
m_counterfeitv3 = widgets.Checkbox(value=False, description='Counterfeit V3.0')
m_aom2 = widgets.Checkbox(value=False, description='AbyssOrangeMix 2 (SFW)')
m_aom3 = widgets.Checkbox(value=False, description='AbyssOrangeMix 3')
m_aom3a1 = widgets.Checkbox(value=False, description='AbyssOrangeMix 3A1')
m_aom3a1b = widgets.Checkbox(value=False, description='AbyssOrangeMix 3A1B')
m_aom3a2 = widgets.Checkbox(value=False, description='AbyssOrangeMix 3A2')
m_aom3a3 = widgets.Checkbox(value=False, description='AbyssOrangeMix 3A3')
m_bom = widgets.Checkbox(value=False, description='BloodOrangeMix')
m_midnightmelt = widgets.Checkbox(value=False, description='Midnight Melt')
m_midnightmaple = widgets.Checkbox(value=False, description='Midnight Maple')
m_midnightmixermelt = widgets.Checkbox(value=False, description='Midnight Mixer Melt')
m_cornflowerv9 = widgets.Checkbox(value=False, description='Cornflower V9')
m_cornflowerx = widgets.Checkbox(value=False, description='Cornflower X')
m_ambientmix = widgets.Checkbox(value=False, description='AmbientMix')
m_hassaku = widgets.Checkbox(value=False, description='Hassaku')
m_grapefruit = widgets.Checkbox(value=False, description='Grapefruit')
m_meinamix = widgets.Checkbox(value=False, description='MeinaMix')
m_meinamix_inpaint = widgets.Checkbox(value=False, description='MeinaMix (inpainting)')
m_meinahentai = widgets.Checkbox(value=False, description='MeinaHentai')
m_meinahentai_inpaint = widgets.Checkbox(value=False, description='MeinaHentai (inpainting)')
m_nai = widgets.Checkbox(value=False, description='NovelAI')
m_naisfw = widgets.Checkbox(value=False, description='NovelAI SFW')
# SD 2
m_sd2 = widgets.Checkbox(value=False, description='Stable Diffusion 2.1')
m_sd2_768 = widgets.Checkbox(value=False, description='Stable Diffusion 2.1 768px')
m_sd2_depth = widgets.Checkbox(value=False, description='Stable Diffusion 2 Depth Model')
m_realengine = widgets.Checkbox(value=False, description='Realism Engine')
m_wd14 = widgets.Checkbox(value=False, description='Waifu Diffusion 1.4')
m_wd15_ink = widgets.Checkbox(value=False, description='Waifu Diffusion 1.5 Ink')
m_wd15_radiance = widgets.Checkbox(value=False, description='Waifu Diffusion 1.5 Radiance')
m_wd15_illusion = widgets.Checkbox(value=False, description='Waifu Diffusion 1.5 Illusion')
m_wd15_mofu = widgets.Checkbox(value=False, description='Waifu Diffusion 1.5 Mofu')
m_illumdif = widgets.Checkbox(value=False, description='Illuminutty Diffusion')
# SD XL
m_sdxl = widgets.Checkbox(value=False, description='Stable Diffusion XL')
m_sdxlrefiner = widgets.Checkbox(value=False, description='Stable Diffusion XL Refiner')
m_counterfeitxl_a = widgets.Checkbox(value=False, description='Counterfeit XL α')
m_counterfeitxl_b = widgets.Checkbox(value=False, description='Counterfeit XL β')
m_dreamshaperxl = widgets.Checkbox(value=False, description='Dreamshaper XL')

sd15_modellist = [m_sd15, m_sd15inpaint, m_realv13, m_realv2, m_chilloutmix, m_deliberate,
                 m_dreamshaperv3, m_dreamshaperv4, m_dreamshaperv5, m_ned, m_openjourney, m_rev,
                 m_perfectworld, m_wd13, m_anythingv3, m_anythingv4, m_anythingv5, m_counterfeitv2,
                 m_counterfeitv3, m_aom2, m_aom3, m_aom3a1, m_aom3a1b, m_aom3a2, m_aom3a3, m_bom,
                 m_midnightmaple, m_midnightmelt, m_midnightmixermelt, m_cornflowerv9, m_cornflowerx,
                 m_ambientmix, m_hassaku, m_grapefruit, m_meinamix, m_meinamix_inpaint, m_meinahentai,
                 m_meinahentai_inpaint, m_nai, m_naisfw]
sd2_modellist = [m_sd2, m_sd2_768, m_sd2_depth, m_illumdif, m_realengine, m_wd14,
                 m_wd15_radiance, m_wd15_ink, m_wd15_mofu, m_wd15_illusion]
sdxl_modellist = [m_sdxl, m_sdxlrefiner, m_counterfeitxl_a, m_counterfeitxl_b, m_dreamshaperxl]
all_modellist = sd15_modellist + sd2_modellist + sdxl_modellist

m_presets = {
    "None": [],
    "Stable Diffusion 1.5": [m_sd15, m_sd15inpaint],
    "Stable Diffusion 2": [m_sd2, m_sd2_768, m_sd2_depth],
    "Anything Model": [m_anythingv3, m_anythingv4, m_anythingv5],
    "Orange Mixes": [m_aom2, m_aom3, m_aom3a1, m_aom3a1b, m_aom3a2, m_aom3a3],
    "Midnight Mixes": [m_midnightmaple, m_midnightmelt, m_midnightmixermelt],
    "Waifu Diffusion 1.5": [m_wd15_radiance, m_wd15_ink, m_wd15_mofu, m_wd15_illusion]
}

m_presets_dropdown = widgets.Dropdown(
    options=list(m_presets.keys()),
    value='None',
    description='Preset:',
    disabled=False,
)

sd15_checkboxes = widgets.GridBox(
    sd15_modellist,
    layout=widgets.Layout(grid_template_columns="repeat(5, 250px)")
)
sd2_checkboxes = widgets.GridBox(
    sd2_modellist,
    layout=widgets.Layout(grid_template_columns="repeat(5, 250px)")
)
sdxl_checkboxes = widgets.GridBox(
    sdxl_modellist,
    layout=widgets.Layout(grid_template_columns="repeat(5, 250px)")
)

m_urldownloadstring = widgets.Textarea(
    value="",
    placeholder="Insert URLs here",
    description="Model URLs:",
    layout=widgets.Layout(width='1500px')
)
m_hftokenstring = widgets.Text(
    value="",
    placeholder="Insert huggingface token",
    description="HF Token:",
    layout=widgets.Layout(width='400px')
)

downloadmodel_button = widgets.Button(
    description='Download',
    disabled=False,
    button_style=''
)

m_downloadhint = """
Check the models you want to download below, or select a preset.
"""
m_downloadhint = m_downloadhint.strip().replace("\n", " ")
m_urlhint = """
Alternatively, fill this with download links (ex: <a href="https://huggingface.co/" target="_blank">huggingface</a>, <a href="https://civitai.com/" target="_blank">civitai</a>).
Download multiple models by separating each link with commas
<br>Insert your huggingface token if you need to download a private model
"""
m_urlhint = m_urlhint.strip().replace("\n", " ")

layout = widgets.VBox([
    widgets.HTML(value=m_downloadhint),
    m_presets_dropdown,
    widgets.HTML(value="<h2>SD 1.5</h2>"),
    sd15_checkboxes,
    widgets.HTML(value="<h2>SD 2</h2>"),
    sd2_checkboxes,
    widgets.HTML(value="<h2>SD XL</h2>"),
    sdxl_checkboxes,
    widgets.HTML(value=m_urlhint),
    m_urldownloadstring,
    m_hftokenstring
])

# URL format: (url, filename, hf_token)
model_links = {
    m_sd15: ("https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt",),
    m_sd15inpaint: ("https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt",),
    m_realv13: ("https://civitai.com/api/download/models/6987",),
    m_realv2: ("https://civitai.com/api/download/models/29460",),
    m_chilloutmix: ("https://civitai.com/api/download/models/11732",),
    m_deliberate: ("https://civitai.com/api/download/models/15236",),
    m_dreamshaperv3: ("https://civitai.com/api/download/models/6202",),
    m_dreamshaperv4: ("https://civitai.com/api/download/models/25067",),
    m_dreamshaperv5: ("https://civitai.com/api/download/models/43888",),
    m_ned: ("https://civitai.com/api/download/models/11925",),
    m_openjourney: ("https://huggingface.co/prompthero/openjourney-v4/resolve/main/openjourney-v4.ckpt",),
    m_rev: ("https://civitai.com/api/download/models/46846",),
    m_perfectworld: ("https://civitai.com/api/download/models/77276",),
    m_wd13: ("https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float16.ckpt",),
    m_anythingv3: ("https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors",),
    m_anythingv4: ("https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned-fp16.ckpt",),
    m_anythingv5: ("https://civitai.com/api/download/models/30163",),
    m_counterfeitv2: ("https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5_fp16.safetensors",),
    m_counterfeitv3: ("https://huggingface.co/gsdf/Counterfeit-V3.0/resolve/main/Counterfeit-V3.0_fp16.safetensors"),
    m_aom2: ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/Pruned/AbyssOrangeMix2_sfw_pruned_fp16_with_VAE.safetensors",),
    m_aom3: ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3_orangemixs.safetensors",),
    m_aom3a1: ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors",),
    m_aom3a1b: ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1B_orangemixs.safetensors",),
    m_aom3a2: ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A2_orangemixs.safetensors",),
    m_aom3a3: ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors",),
    m_bom: ("https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/BloodOrangeMix/BloodOrangeMix.safetensors",),
    m_midnightmaple: ("https://huggingface.co/DrBob2142/Midnight_Mixes/resolve/main/Midnight%20Maple.safetensors",),
    m_midnightmelt: ("https://huggingface.co/DrBob2142/Midnight_Mixes/resolve/main/Midnight%20Melt.safetensors",),
    m_midnightmixermelt: ("https://huggingface.co/DrBob2142/Midnight_Mixes/resolve/main/Midnight%20Mixer%20Melt.safetensors",),
    m_cornflowerv9: ("https://civitai.com/api/download/models/11690",),
    m_cornflowerx: ("https://civitai.com/api/download/models/39123",),
    m_ambientmix: ("https://civitai.com/api/download/models/31866",),
    m_hassaku: ("https://civitai.com/api/download/models/62528",),
    m_grapefruit: ("https://civitai.com/api/download/models/29179",),
    m_meinamix: ("https://civitai.com/api/download/models/119057",),
    m_meinamix_inpaint: ("https://civitai.com/api/download/models/120702",),
    m_meinahentai: ("https://civitai.com/api/download/models/100675",),
    m_meinahentai_inpaint: ("https://civitai.com/api/download/models/119992",),
    m_nai: ("https://huggingface.co/Chronophobia/animefull/resolve/main/nai-fp16.safetensors",),
    m_naisfw: ("https://huggingface.co/Chronophobia/animefull/resolve/main/naisfw-fp16.safetensors",),
    m_sd2: ("https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt", ("https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml", "v2-1_512-ema-pruned.yaml")),
    m_sd2_768: ("https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt", ("https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml", "v2-1_768-ema-pruned.yaml")),
    m_sd2_depth: ("https://huggingface.co/stabilityai/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt", ("https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-midas-inference.yaml", "512-depth-ema.yaml")),
    m_realengine: ("https://civitai.com/api/download/models/20414", "https://civitai.com/api/download/models/20414?type=Config"),
    m_illumdif: ("https://civitai.com/api/download/models/42304","https://civitai.com/api/download/models/42304?type=Config"),
    m_wd14: ("https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/wd-1-4-anime_e1.ckpt", "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/wd-1-4-anime_e1.yaml"),
    m_wd15_ink: ("https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-ink-fp16.safetensors",),
    m_wd15_illusion: ("https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-illusion-fp16.safetensors",),
    m_wd15_radiance: ("https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-radiance-fp16.safetensors",),
    m_wd15_mofu: ("https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-mofu-fp16.safetensors",),
    m_sdxl: ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors",),
    m_sdxlrefiner: ("https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors",),
    m_counterfeitxl_a: (("https://huggingface.co/gsdf/CounterfeitXL/resolve/main/CounterfeitXL_%CE%B1.safetensors","CounterfeitXL_α.safetensors"),),
    m_counterfeitxl_b: (("https://huggingface.co/gsdf/CounterfeitXL/resolve/main/CounterfeitXL_%CE%B2.safetensors","CounterfeitXL_β.safetensors"),),
    m_dreamshaperxl: ("https://civitai.com/api/download/models/126688",)
}

def selectModelPresets(change):
  selected_preset = change['new']
  checked_list = m_presets[selected_preset]
  for checkbox in all_modellist:
    if checkbox in checked_list:
      checkbox.value = True
    else:
      checkbox.value = False

m_presets_dropdown.observe(selectModelPresets, names='value')

display(layout)

In [ ]:
#@title ## Download Checkpoints
#@markdown Run this cell once you configured the above cell.

vae_list = [
    ("https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt", "vae-ft-mse-840000-ema-pruned.ckpt"),
    ("https://huggingface.co/Chronophobia/animefull/resolve/main/animevae.pt", "animevae.pt"),
    ("https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt", "kl-f8-anime.ckpt"),
    ("https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt", "kl-f8-anime2.ckpt")
]

def downloadModelURL(url, hf_token=None):
  filename = None
  if isinstance(url, (list, tuple)):
    model_url = url[0]
    try:
      filename = url[1]
    except IndexError:
      pass
    try:
      hf_token = url[2]
    except IndexError:
      pass
  else:
    model_url = url
  if not filename:
    filename = model_url.split('/')[-1].replace('%20', ' ') if "huggingface.co" in model_url else None

  if hf_token and "huggingface.co" in model_url:
    user_header = f'"Authorization: Bearer {hf_token}"'
    auth = f'--header={user_header}'
  else:
    auth = ''
  if filename:
    output = f'-o "{filename}"'
  else:
    output = ''

  !aria2c --console-log-level=error {auth} -c -x 16 -s 16 -k 1M {model_url} {output}

def downloadModels():
  m_downloadlist = []
  m_hftoken = m_hftokenstring.value if m_hftokenstring.value else None
  for checkbox in all_modellist:
    if checkbox.value:
      m_downloadlist.extend(model_links[checkbox])
  if m_urldownloadstring.value:
    for model in m_urldownloadstring.value.strip().split(","):
      m_downloadlist.append(model)
  %cd {userfiles_dir}/models/Stable-diffusion
  download_amount = len(m_downloadlist)
  download_left = 1
  print("---- Downloading SD models ----")
  for url in m_downloadlist:
    print(f"Downloading {download_left}/{download_amount}...")
    downloadModelURL(url, m_hftoken)
    download_left += 1

downloadModels()

def downloadVAE(vae):
  url = vae[0]
  filename = vae[1]
  if not os.path.exists(f"{webui_dir}/models/vae/{filename}"):
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {userfiles_dir}/models/VAE -o {filename}

!mkdir -p {userfiles_dir}/models/VAE
for vae in vae_list:
  downloadVAE(vae)

# 2.2. (Optional) Install Extensions

In [ ]:
#@title ## Install Extensions
ControlNet = False #@param {type:"boolean"}
ControlNet_model = "All" #@param ["All", "Canny", "MLSD", "Depth", "Normal", "Segmentation", "Inpaint", "Lineart", "Lineart Anime", "Openpose", "Scribble", "Soft Edge", "Shuffle", "Instruct Pix2Pix", "Tile"]
Additional_Networks = False #@param {type:"boolean"}
WebUI_Cutoff = False #@param {type:"boolean"}
MultiDiffusion = False #@param {type:"boolean"}
UltimateSDUpscale = False #@param {type:"boolean"}
Dynamic_Thresholding = False #@param {type:"boolean"}
Aesthetic_Gradients = False #@param {type:"boolean"}
Self_Attention_Guidance = False #@param {type:"boolean"}
System_Info = False #@param {type:"boolean"}
CivitaiHelper = False #@param {type:"boolean"}
LoRA_Block_Weight = False #@param {type:"boolean"}
CanvasZoom = False #@param {type:"boolean"}
ModelToolkit = False #@param {type:"boolean"}
SuperMerger = False #@param {type:"boolean"}
Infinity_Grid_Generator = False #@param {type:"boolean"}
Extensions_from_URL = "" #@param {type:"string"}

LyCORIS = False
#@markdown #### Fill this to install extensions not listed above. Can type the full link or use this format: `user/repo`
webui
import json, requests

ext_list = [
    ControlNet, Additional_Networks, UltimateSDUpscale, LyCORIS,
    Aesthetic_Gradients, CivitaiHelper, CanvasZoom, System_Info,
    Dynamic_Thresholding, ModelToolkit, SuperMerger, WebUI_Cutoff,
    MultiDiffusion, Self_Attention_Guidance, LoRA_Block_Weight, Infinity_Grid_Generator
            ]
ext_install = []

controlnet_models = {
    "Canny": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth",
    "MLSD": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth",
    "Depth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth",
    "Normal": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth",
    "Segmentation": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth",
    "Inpaint": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth",
    "Lineart": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth",
    "Lineart Anime": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth",
    "Openpose": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth",
    "Scribble": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth",
    "Soft Edge": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth",
    "Shuffle": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth",
    "Instruct Pix2Pix": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth",
    "Tile": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth"
}

t2iadapter_models = {
    "Color": "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth",
    "Style": "https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth"
}

ext_repojson = requests.get("https://github.com/Khronophobia/stable-diffusion-Colab/raw/main/ext_repoindex.json")
ext_repoindex = json.loads(ext_repojson.text)

def readExtensionsList(file_path):
  with open(file_path, 'r') as file:
    contents = file.readlines()
    contents = [line.strip() for line in contents if line.strip() and not line.startswith('#')]
    return contents

for index, ext in enumerate(ext_list):
  if ext:
    ext_install.append(ext_repoindex[str(index)])

if driveIsMounted():
  extlist_file = output_path + '/extensions_list.txt'
  ext_install.extend([(ext, None) for ext in readExtensionsList(extlist_file)])

ext_install.extend([(ext.strip(), None) for ext in Extensions_from_URL.split(',') if Extensions_from_URL])

def downloadControlNetModels(modelname, modellist):
  model = modellist[modelname]
  #config = model.replace('.pth', '.yaml')
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model} -o {model.split('/')[-1]}
  #!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {config} -o {config.split('/')[-1]}

def installExtensions(ext, ext_displayname=None):
  ext = ext.replace(";", "|")
  ext_commit = None
  ext_link, _, ext_commit = ext.strip().partition("|")
  if not ext_link.startswith("https://") and not ext_link.startswith("http://"):
    if not "github.com" in ext_link:
      ext_link = "github.com/" + ext_link
    ext_link = "https://" + ext_link
  ext_link = ext_link.replace("http://", "https://")
  if not ext_link.endswith('.git'):
    ext_link += '.git'
  ext_name = ext_link.split("/")[-1].split(".")[0]

  print(f"Installing {ext_displayname}...") if ext_displayname else print(f"Installing {ext_name}...")
  !git clone {ext_link}
  %cd {ext_name}
  if ext_commit:
    !git checkout -f {ext_commit}
  print(ext_name, "Install completed.")

!mkdir -p {userfiles_dir}/extensions
if ext_install:
  for ext in ext_install:
    if ext[1] == "Canvas Zoom" and webui_repo == "UI-UX":
      print("Skipping Canvas Zoom because it's unneeded in UI-UX.")
      continue
    %cd {userfiles_dir}/extensions
    installExtensions(ext[0], ext[1])
    if ext[1] == "ControlNet":
      %cd {userfiles_dir}/extensions/sd-{unforgiveable}-controlnet/models
      if ControlNet_model == "All":
        for model in controlnet_models:
          downloadControlNetModels(model, controlnet_models)
      else:
        downloadControlNetModels(ControlNet_model)
      for model in t2iadapter_models:
        downloadControlNetModels(model, t2iadapter_models)
      !sudo apt install libcairo2-dev pkg-config python3-dev
    if ext[1] == "Aesthetic Gradients" and driveIsMounted():
      !mkdir -p {output_path}/aesthetic_embeddings
      %cd {userfiles_dir}/extensions/stable-diffusion-{unforgiveable}-aesthetic-gradients
      !rm -rf aesthetic_embeddings
      !ln -s {output_path}/aesthetic_embeddings
    if ext[1] == "LoRA Block Weight" and driveIsMounted():
      !mkdir -p {output_path}/extensions-config/sd-{unforgiveable}-lora-block-weight
      if not os.path.exists(output_path + f'/extensions-config/sd-{unforgiveable}-lora-block-weight/lbwpresets.txt'):
        !wget -qq https://github.com/Khronophobia/stable-diffusion-Colab/raw/main/extension-configs/lbwpresets.txt -O {output_path}/extensions-config/sd-{unforgiveable}-lora-block-weight/lbwpresets.txt
      !ln -s {output_path}/extensions-config/sd-{unforgiveable}-lora-block-weight/lbwpresets.txt {userfiles_dir}/extensions/sd-{unforgiveable}-lora-block-weight/scripts

def linkExtensionstoWebUI():
  %cd {webui_dir}
  !rm -rf {webui_dir}/extensions
  !ln -s {userfiles_dir}/extensions

linkExtensionstoWebUI()

# 3. Run the WebUI

In [ ]:
#@title ## Run this cell and wait until a Gradio/NGROK link appear, then click it to open the WebUI
username = "" #@param {type:"string"}
password = "" #@param {type:"string"}
#@markdown ##### (Optional) Adds a login page.
NGROK = "" #@param {type:"string"}
#@markdown ##### (Optional) Fill this with your NGROK authtoken. Get them from [here.](https://dashboard.ngrok.com/get-started/your-authtoken)
args = "--no-half-vae" #@param {type:"string"}
#@markdown ##### Fill this with launch arguments as needed

default_args = f"""
--ckpt-dir {userfiles_dir}/models/Stable-diffusion
--vae-dir {userfiles_dir}/models/VAE
--enable-insecure-extension-access
--xformers
"""
default_args = default_args.strip().replace("\n", " ")

if NGROK:
  share = f"--ngrok {NGROK}"
else:
  share = "--share"
if username and password:
  auth = f"--gradio-auth {username}:{password}"
else:
  auth = ""
%cd {webui_dir}
!python launch.py {default_args} {args} {share} {auth}